<a href="https://colab.research.google.com/github/rosscampbell7/test-repo/blob/master/WandB_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb

In [10]:
# Define the neural network model
class SimpleCNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 100)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



In [14]:
def train():
    # Initialize wandb with the current run
    wandb.init()

    # Hyperparameters
    batch_size = wandb.config.batch_size
    learning_rate = wandb.config.learning_rate
    num_epochs = 1#wandb.config.epochs
    optimizer_type = wandb.config.optimizer
    momentum = wandb.config.momentum
    weight_decay = wandb.config.weight_decay
    dropout_rate = wandb.config.dropout_rate
    scheduler_type = wandb.config.scheduler

    # Check if GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data augmentation and normalization for training
    transform = transforms.Compose(
        [transforms.RandomHorizontalFlip(),
         transforms.RandomCrop(32, padding=4),
         transforms.ToTensor(),
         transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))])

    # Load CIFAR100 dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Initialize the model, loss function, and optimizer
    model = SimpleCNN(dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_type == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

    if scheduler_type == 'StepLR':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    elif scheduler_type == 'ExponentialLR':
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    else:
        scheduler = None

    # Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        model.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if i % 100 == 99:  # Log every 100 mini-batches
                accuracy = 100 * correct / total
                wandb.log({"loss": running_loss / 100, "accuracy": accuracy, "epoch": epoch + 1})
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}, accuracy: {accuracy:.2f}%")
                running_loss = 0.0
                correct = 0
                total = 0

        if scheduler:
            scheduler.step()

    print("Finished Training")
    torch.save(model.state_dict(), "cifar100_model.pth")

if __name__ == "__main__":
    # Sweep configuration
    sweep_config = {
        'method': 'random',  # Random search
        'parameters': {
            'batch_size': {
                'values': [32, 64, 128]
            },
            'learning_rate': {
                'values': [0.1, 0.01, 0.001]
            },
            'optimizer': {
                'values': ['SGD', 'Adam', 'RMSprop']
            },
            'momentum': {
                'values': [0.9, 0.95, 0.99]
            },
            'weight_decay': {
                'values': [0.0001, 0.001, 0.01]
            },
            'dropout_rate': {
                'values': [0.3, 0.5, 0.7]
            },
            'scheduler': {
                'values': ['None', 'StepLR', 'ExponentialLR']
            }
        }
    }


    # Initialize the sweep
    sweep_id = wandb.sweep(sweep_config, project="cifar100-classifier")

    # Start the sweep
    wandb.agent(sweep_id, function=train,)

Create sweep with ID: sdws3n5m
Sweep URL: https://wandb.ai/themachine/cifar100-classifier/sweeps/sdws3n5m


wandb: Agent Starting Run: nqi5r5z5 with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.7
wandb: 	learning_rate: 0.1
wandb: 	momentum: 0.95
wandb: 	optimizer: SGD
wandb: 	scheduler: None
wandb: 	weight_decay: 0.0001


Files already downloaded and verified
Files already downloaded and verified
[1, 100] loss: 2.924, accuracy: 10.36%
[1, 200] loss: 2.392, accuracy: 10.39%
[1, 300] loss: 2.368, accuracy: 9.91%
[1, 400] loss: 2.348, accuracy: 10.30%
[1, 500] loss: 2.348, accuracy: 10.14%
[1, 600] loss: 2.336, accuracy: 10.20%
[1, 700] loss: 2.334, accuracy: 9.81%
Finished Training


accuracy,██▂▇▅▆▁
epoch,▁▁▁▁▁▁▁
loss,█▂▁▁▁▁▁
accuracy,9.8125
epoch,1
loss,2.33367


wandb: Agent Starting Run: lht4vvef with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.3
wandb: 	learning_rate: 0.1
wandb: 	momentum: 0.9
wandb: 	optimizer: SGD
wandb: 	scheduler: None
wandb: 	weight_decay: 0.01


...

accuracy,▁▆█
epoch,▁▁▁
loss,█▃▁
accuracy,40.04688
epoch,1
loss,1.64169


wandb: Agent Starting Run: lr0zlpvq with config:
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.7
wandb: 	learning_rate: 0.1
wandb: 	momentum: 0.95
wandb: 	optimizer: RMSprop
wandb: 	scheduler: ExponentialLR
wandb: 	weight_decay: 0.0001


Files already downloaded and verified
Files already downloaded and verified
[1, 100] loss: 873157.970, accuracy: 9.62%
[1, 200] loss: 2.426, accuracy: 10.00%
[1, 300] loss: 333.389, accuracy: 9.41%
[1, 400] loss: 4449.225, accuracy: 10.28%
[1, 500] loss: 2.336, accuracy: 10.59%
[1, 600] loss: 2.396, accuracy: 8.84%
[1, 700] loss: 39.985, accuracy: 10.09%
[1, 800] loss: 2.338, accuracy: 8.78%
[1, 900] loss: 2.352, accuracy: 11.03%
[1, 1000] loss: 17.029, accuracy: 10.75%
[1, 1100] loss: 2.371, accuracy: 9.78%
[1, 1200] loss: 2.371, accuracy: 10.41%
[1, 1300] loss: 50.836, accuracy: 10.16%
[1, 1400] loss: 2.386, accuracy: 8.91%
[1, 1500] loss: 2.356, accuracy: 9.31%
Finished Training


accuracy,▄▅▃▆▇▁▅▁█▇▄▆▅▁▃
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,9.3125
epoch,1
loss,2.35582


wandb: Ctrl + C detected. Stopping sweep.


In [18]:
# Log the model as a wandb artifact
artifact = wandb.Artifact('cifar100', type='model')
artifact.add_file('/content/wandb/')
wandb.log_artifact(artifact)

ValueError: Path is not a file: '/content/wandb/'

In [ ]:
wandb.finish()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')